In [1]:
try:
  # We must install required packages if we are in Google Colab
  import google.colab
  %pip install roboticstoolbox-python>=1.0.2
  %pip install sympy
except:
  # We are not in Google Colab
  # Apply custon style to notebook
  from IPython.core.display import HTML
  import pathlib
  styles_path = pathlib.Path(pathlib.Path().absolute(), "style", "style.css")
  styles = open(styles_path, "r").read()
  HTML(f"<style>{styles}</style>")

# 3.0 Analytic Forms

$\large{\text{Manipulator Differential Kinematics}} \\ \large{\text{Part II: Acceleration and Advanced Applications}}$

$\text{By Jesse Haviland and Peter Corke}$

<br>

The sections of the Tutorial paper related to this notebook are listed next to each contents entry.
It is beneficial to read these sections of the paper before attempting the notebook Section.

We start with some important background theory on the Special Orthogonal Group in Sections 3.1 - 3.4. Skip ahead to Section 3.5 if you want to go straight to the analytical Jacobian.

### Contents

[3.1 Lie Algebra and the Tangent Space – $\bf{so}(3)$](#so3t)

[3.2 Special Orthogonal Group – $\bf{SO}(3)$ ](#so31)

[3.3 Derivatives of $\bf{SO}(3)$ and the Angular Jacobian](#so3d)

[3.4 Second Derivatives of $\bf{SO}(3)$ and the Angular Hessian](#acc)

[3.5 The Analytic Jacobian](#Ja)
* Analytical Form

[3.6 Derivative of the Analytic Jacobian](#Jad)
* Analytical Form

In [2]:
# We will do the imports required for this notebook here

# numpy provides import array and linear algebra utilities
import numpy as np

# the robotics toolbox provides robotics specific functionality
import roboticstoolbox as rtb

# spatial math provides objects for representing transformations
import spatialmath as sm
import spatialmath.base as smb

# sympy provides symbolic mathematic operations 
import sympy as sym

# the math module
import math

# skew and vex operations (see Part 1 Notebook 2)
from spatialmath.base import skew, vex

# The matrix exponential method
from scipy.linalg import expm

# setting up sympy to print nicely
from IPython import display as ds
from sympy.physics.mechanics import init_vprinting
from sympy.physics.vector import vlatex
init_vprinting(use_latex='mathjax')

<br>

<a id='so3t'></a>

### 3.1 Lie Algebra and the Tangent Space – $\bf{so}(3)$
---


<p class="asasasas">

Rotations in 3-dimensions can be represented by matrices which form Lie groups and which have Lie algebras. These rotations form the Special Orthogonal group in 3 dimensions, denoted as $\bf{SO}(3)$.

</p>

An implication of being a Lie group is that there is a smooth and differentiable manifold structure. At any point on the manifold we can construct tangent vectors. The set of all tangent vectors at that point form a vector space - the tangent space. This is the multidimensional equivalent to a tangent line on a curve, or a tangent plane on a solid. We can think of this as the set of all possible derivatives of the manifold at that point.

The tangent space of $\bf{SO}(3)$ lies in $\bf{so}(3)$ which are the Lie algebras.

The tangent space at the identity is described by the Lie algebra of the group, and the basis directions of the tangent space are called the generators of the group. There are three generators of $\bf{SO}(3)$ each belonging to $\bf{so}(3)$.

The three generators each provide a basis – which can be thought of as the $x$, $y$, and $z$ axes – for the group $\bf{SO}(3)$.

The generator for the x-axis is
\begin{equation*}
    g_x =
    \begin{pmatrix}
        0 & 0 & 0 \\
        0 & 0 & -1 \\
        0 & 1 & 0
    \end{pmatrix}.
\end{equation*}

The generator for the y-axis is
\begin{equation*}
    g_y =
    \begin{pmatrix}
        0 & 0 & 1 \\
        0 & 0 & 0 \\
        -1 & 0 & 0
    \end{pmatrix}.
\end{equation*}

The generator for the z-axis is
\begin{equation*}
    g_z =
    \begin{pmatrix}
        0 & -1 & 0 \\
        1 & 0 & 0 \\
        0 & 0 & 0
    \end{pmatrix}.
\end{equation*}

Every point in the tangent space $\bf{so}(3)$ – the derivatives of the manifold $\bf{SO}(3)$ – can be expressed as a linear combination of basis matrices

\begin{equation*}
    \Omega(\eta_1, \ \eta_2, \ \eta_3) =
    \eta_1 \ g_x +
    \eta_2 \ g_y +
    \eta_3 \ g_z
\end{equation*}
where $\bf{\eta} = (\eta_1, \ \eta_2, \ \eta_3)$ are the exponential coordinates (also known as the Euler vector).

Together, the exponential coordinates define an axis and an angle

\begin{equation*}
    \bf{\eta} =
    \hat{\eta} \ \theta
\end{equation*}
where $\hat{\eta} \in \mathbb{R}^3$ is a unit vector which defines the axis and $\theta$ defines the angle or rotation around the axis.

Therefore $\eta$ describes four numbers with
\begin{equation*}
    \theta = \lvert\lvert \eta \rvert\rvert
\end{equation*}
and 
\begin{equation*}
    \hat{\eta} = \dfrac{\eta}{\lvert\lvert \eta \rvert\rvert}
\end{equation*}



In [3]:
# Lets make the basis matrices in Python
gx = smb.skew([1, 0, 0])
gy = smb.skew([0, 1, 0])
gz = smb.skew([0, 0, 1])

# Visualise the results
ds.display(ds.Math(f"g_x = {sym.latex(sym.Matrix(gx))}"))
ds.display(ds.Math(f"g_y = {sym.latex(sym.Matrix(gy))}"))
ds.display(ds.Math(f"g_z = {sym.latex(sym.Matrix(gz))}"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

We have the following relationship between the Lie algebra $\phi$ and Lie group $\Phi$

\begin{equation*}
    \forall \bf{X} \in \bf{\phi} \Rightarrow e^{\bf{X}} \in \bf{\Phi}
\end{equation*}
which says that the exponential of any member of $\bf{so}(3)$ is a valid member of $\bf{SO}(3)$

Lets put this to the test in Python.

We will use the Spatialmath Package to produce a rotation in $\bf{SO}(3)$ which is around the x-axis by an amount $\pi$

In [4]:
rot_x = sm.SO3.Rx(np.pi).A

# View the result
ds.display(ds.Math(f"R_x(\pi) = {sym.latex(sym.Matrix(rot_x))}"))


<IPython.core.display.Math object>

Now we we get the same rotation using the theory detailed out above

We will perform the equation
\begin{align*}
    R_x(\theta) &= e^{\theta \ g_x} \\
    R_x(\pi) &= e^{\pi \ g_x}
\end{align*}

In [5]:
rot_x = expm(np.pi * gx)

# View the result
ds.display(ds.Math(f"R_x(\pi) = {sym.latex(sym.Matrix(rot_x))}"))

<IPython.core.display.Math object>

which shows that we get the same result.

<br>

<a id='so31'></a>

### 3.2 Special Orthogonal Group – $\bf{SO}(3)$ 
---

We can create every rotation in $\bf{SO}(3)$ with the generators

\begin{align*}
    \Omega(\bf\eta)
    &= 
    \begin{pmatrix}
        0 & -\eta_3 & \eta_2 \\
        \eta_3 & 0 & -\eta_1 \\
        -\eta_2 & \eta_1 & 0
    \end{pmatrix} \\
    &= 
    [\eta]_\times
\end{align*}
where $[\cdot]_\times : \mathbb{R}^3 \mapsto \bf{so}(3)$ maps a vector to a skew symmetric matrix.

We then create a rotation with

\begin{align*}
    \bf{R}(\bf\eta)
    &= e^{[\eta]_\times} \\
    &= e^{[\hat{\eta}]_\times \ \theta} \\
\end{align*}


Coming up with the axis and angle to define $\eta$ can be tricky in practice. Another strategy can be to construct rotations using a combination of the generators.

For example, we can construct a rotation made from roll-pitch-yaw angles applied in the xyz order:
\begin{align*}
    \bf{R}(\alpha, \ \beta, \ \gamma)
    &= 
    e^{\gamma \ g_x} \ e^{\beta \ g_y} \ e^{\alpha \ g_y}
\end{align*}

Lets view a rotation with rpy angles $\Gamma = (\frac{\pi}{2}, \ \frac{\pi}{2}, \ \frac{\pi}{2})$ made using the Spatialmaths package

In [6]:
rot_rpy = sm.SO3.RPY([np.pi/2, np.pi/2, np.pi/2], order='xyz').A

# View the result
ds.display(ds.Math(f"R = {sym.latex(sym.Matrix(rot_rpy))}"))

<IPython.core.display.Math object>

Now lets try using the generators

In [7]:
rot_rpy = expm(np.pi/2 * gx) @ expm(np.pi/2 * gy) @ expm(np.pi/2 * gz)

# View the result
ds.display(ds.Math(f"R = {sym.latex(sym.Matrix(rot_rpy))}"))

<IPython.core.display.Math object>

<br>

<a id='so3d'></a>

### 3.3 Derivatives of $\bf{SO}(3)$ and the Angular Jacobian
---

In this section, we are going to define a new Jacobian which relates some angular rate unit to angular velocity. In this case we are choosing xyz roll-pitch-yaw angle rates. Using the same techniques presented in this Notebook, you can create a new Jacobian to relate different angular rate units if desired.

We have the following formula for a member of $\bf{SO}(3)$

\begin{align*}
    \bf{R}(\bf\eta)
    &= e^{[\eta]_\times} \\
\end{align*}

Using the chain rule, we can take the derivative

\begin{align*}
    \dot{\bf{R}}
    &= 
    \dfrac{\mathrm{d}\ {[\eta]_\times}}{\mathrm{d} t}
    \ \
    e^{[\eta]_\times} \\
    &= 
    [\omega]_\times \
    e^{[\eta]_\times} \\
    &= 
    [\omega]_\times \
    \bf{R}
\end{align*}
where $\omega = (\omega_x, \ \omega_y, \ \omega_z)$ describes the angular velocity. In a similar theme to $\eta$, the angular velocity describes an axis of rotation, and a rate of rotation about that axis.

To test some derivatives, lets first make some symbolic variables in Python using the `sympy` package. These variables will define some roll-pitch-yaw angles $\Gamma = (\alpha, \ \beta, \ \gamma)$

In [8]:
# A variable for time
t = sym.symbols('t')

# Variables for roll, pitch and yaw
α, β, γ = sym.symbols("α β γ")

# Make the angles functions of time
αt = sym.Function(α)(t)
βt = sym.Function(β)(t)
γt = sym.Function(γ)(t)

# List to hold python friendly values
Γ_py = [α, β, γ]

# List to hold the angles
Γ = [αt, βt, γt]

# View the results
ds.display(ds.Math(f"Γ = {sym.latex(Γ)}"))

<IPython.core.display.Math object>

In [9]:
# Now lets makes the derivatives of those angles
# These variables represent the roll, pitch, and yaw rates
αdt = sym.diff(αt)
βdt = sym.diff(βt)
γdt = sym.diff(γt)
Γd_py = [str(a) + "d" for a in Γ_py]
Γd = [αdt, βdt, γdt]

# Now lets makes the derivatives of those velocities
# These variables represent the roll, pitch, and yaw accelerations
αddt = sym.diff(αdt)
βddt = sym.diff(βdt)
γddt = sym.diff(γdt)
Γdd_py = [str(a) + "d" for a in Γd_py]
Γdd = [αddt, βddt, γddt]

# View the results
ds.display(ds.Math(f"\dot{{Γ}} = {vlatex(Γd)}"))
ds.display(ds.Math(f"\ddot{{Γ}} = {vlatex(Γdd)}"))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

The generators are not the only way to construct a rotation. We can use a known identity to construct a rotation around the x, y, or z-axis.

These identities are

$$
    \bf{R}_x(\theta) = 
    \left[\begin{matrix}1 & 0 & 0\\0 & \cos{\left(\theta \right)} & - \sin{\left(\theta \right)}\\0 & \sin{\left(\theta \right)} & \cos{\left(\theta \right)}\end{matrix}\right]
$$

$$
    \bf{R}_y(\theta) = 
    \left[\begin{matrix}\cos{\left(\theta \right)} & 0 & \sin{\left(\theta \right)}\\0 & 1 & 0\\- \sin{\left(\theta \right)} & 0 & \cos{\left(\theta \right)}\end{matrix}\right]
$$

$$
    \bf{R}_z(\theta) = 
    \left[\begin{matrix}\cos{\left(\theta \right)} & - \sin{\left(\theta \right)} & 0\\\sin{\left(\theta \right)} & \cos{\left(\theta \right)} & 0\\0 & 0 & 1\end{matrix}\right]
$$

The Spatialmath package uses these identities to construct $\textbf{SO}(3)$ objects.

For example:

In [10]:
# Make a theta symbolic variable
θ = sym.symbols('θ')

# Make the rotations using the spatialmaths package
rot_x = sm.SO3.Rx(θ).A
rot_y = sm.SO3.Ry(θ).A
rot_z = sm.SO3.Rz(θ).A

# View the results, these should match the equations above
ds.display(ds.Math(f"R_x(θ) = {sym.latex(sym.Matrix(rot_x))}"))
ds.display(ds.Math(f"R_y(θ) = {sym.latex(sym.Matrix(rot_y))}"))
ds.display(ds.Math(f"R_z(θ) = {sym.latex(sym.Matrix(rot_z))}"))

# print(sym.latex(sym.Matrix(rot_x), mode='equation*'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

As we showed previously, a rotation can be constructed by applying combinations of elementary rotations. A rotation can be constructed from roll, pitch, and yaw angles and an order. The order defines which rotations correspond to which rotation.

For example, for zyx roll-pitch-yaw angles, where zyx is the order, and the angles are $\Gamma = (\alpha, \ \beta, \ \gamma)$ the resulting rotation is 

\begin{equation*}
    \bf{R}(\Gamma) = 
    \bf{R}_z(\gamma)
    \bf{R}_y(\beta)
    \bf{R}_x(\alpha)
\end{equation*}

Another example, for xyz roll-pitch-yaw angles, the resulting rotation is 

\begin{equation*}
    \bf{R}(\Gamma) = 
    \bf{R}_x(\gamma)
    \bf{R}_y(\beta)
    \bf{R}_y(\alpha)
\end{equation*}

We will stick with the xyz order for the rest of the this notebook.

Lets make this rotation using Spatialmaths

In [11]:
# Make the elementary rotations
Rx = sm.SO3.Rx(γt).A
Ry = sm.SO3.Ry(βt).A
Rz = sm.SO3.Rz(αt).A

# Make the xyz roll-pitch-yaw rotation
R = Rx @ Ry @ Rz

# View the result
ds.display(ds.Math(f"R(Γ) = {sym.latex(sym.Matrix(R))}"))

<IPython.core.display.Math object>

Now lets take the derivative of $\bf{R}(\Gamma)$ using the product rule

\begin{equation*}
    \dot{\bf{R}} = 
    \dot{\bf{R}_x}
    \bf{R}_y
    \bf{R}_y +
    \bf{R}_x
    \dot{\bf{R}_y}
    \bf{R}_y +
    \bf{R}_x
    \bf{R}_y
    \dot{\bf{R}_y}
\end{equation*}

Now to work out $\dot{\bf{R}_x}$ lets look at the definition of $\bf{R}$

\begin{align*}
    \bf{R}_x(\bf\gamma)
    &= e^{\gamma \ g_x} \\
\end{align*}

and take the derivative using the chain rule as we did above

\begin{align*}
    \dot{\bf{R}}_x
    &= 
    \dfrac{\mathrm{d}\ {\gamma  g_x}}{\mathrm{d} t}
    \ \
    e^{\gamma  g_x} \\
    &= 
    \dot{\gamma}  g_x \
    e^{\gamma  g_x} \\
    &= 
    \dot{\gamma}  g_x \
    \bf{R}_x
\end{align*}

Lets try this out in Python before going any further

In [12]:
# Calculate the derivative of Rx
dRx = (γdt * gx) @ Rx

# View the result
ds.display(ds.Math(f"\dot{{R}}_x = {vlatex(sym.Matrix(dRx))}"))

<IPython.core.display.Math object>

To validate our method, lets use `sympy` symbolic derivative function to compare results

In [13]:
# Use symbolic differentiation
dRx_sym = sym.diff(sym.Matrix(Rx))

# View the sresult
ds.display(ds.Math(f"\dot{{R}}_x = {vlatex(sym.Matrix(dRx_sym))}"))

<IPython.core.display.Math object>

Lets calculate the derivatives for Ry and Rz in the same manner before going any further

In [14]:
# Calculate the derivative of Ry
dRy = (βdt * gy) @ Ry
dRz = (αdt * gz) @ Rz

# View the result
ds.display(ds.Math(f"\dot{{R}}_y = {vlatex(sym.Matrix(dRy))}"))
ds.display(ds.Math(f"\dot{{R}}_z = {vlatex(sym.Matrix(dRz))}"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Now recall our derivative of $\bf{R}(\Gamma)$ previously defined

\begin{equation*}
    \dot{\bf{R}} = 
    \dot{\bf{R}_x}
    \bf{R}_y
    \bf{R}_y +
    \bf{R}_x
    \dot{\bf{R}_y}
    \bf{R}_y +
    \bf{R}_x
    \bf{R}_y
    \dot{\bf{R}_y}
\end{equation*}

While we could simplify this further, this will do for the purpose of this tutorial.

We now have the necessary components to calculate this in Python

In [15]:
dR = dRx @ Ry @ Rz + Rx @ dRy @ Rz + Rx @ Ry @ dRz

# View the result
ds.display(ds.Math(f"\dot{{R}} = {vlatex(sym.Matrix(dR))}"))

<IPython.core.display.Math object>

Now using the identity we derived previously
\begin{align*}
    \dot{\bf{R}}
    &= 
    [\omega]_\times \
    \bf{R}
\end{align*}

we can rearrange to obtain the angular velocity $\omega$

\begin{align*}
    \dot{\bf{R}} \bf{R}^\top
    &= 
    [\omega]_\times \\
    \vee_\times \left( \dot{\bf{R}} \bf{R}^\top \right)
    &= 
    \omega
\end{align*}

Now lets try in Python

In [16]:
ω = vex(dR @ R.T)

# Simplify the result to make it more readable
ω = sym.simplify(ω)

# View the result
ds.display(ds.Math(f"ω = {vlatex(sym.Matrix(ω))}"))

<IPython.core.display.Math object>

We now have a relationship between angular velocity and roll-pitch-yaw rates.

Lets factorise the roll-pitch-yaw rates out of the above result

In [17]:
# Lets factorise the result to extract the roll, pitch and yaw rates
A = sym.Matrix.zeros(3,3)
for i in range(3):
    e = ω[i].expand()
    for j in range(3):
        A[i, j] = e.coeff(Γd[j])

# View the result
ds.display(ds.Math(f"A = {vlatex(sym.Matrix(A))}"))

<IPython.core.display.Math object>

Now we have the matrix $A$. This is a Jacobian which in this case relates angular velocity to xyz roll-pitch-yaw rates

\begin{align*}
    \omega =
    \bf{A}(\Gamma)
    \dot{\Gamma}
\end{align*}

In [18]:
# View in expanded form in Python
ds.display(ds.Math(f"ω = {vlatex(sym.Matrix(A))}{vlatex(sym.Matrix(Γd))} "))

<IPython.core.display.Math object>

This process shows how we get to Equation (31) in the Tutorial. 

We will first create the $\bf{A}^{-1}$ matrix

In [19]:
Ai = sym.trigsimp(A.inv())

# View the result
ds.display(ds.Math(f"A^{{-1}} = {vlatex(sym.Matrix(Ai))}"))

<IPython.core.display.Math object>

Lets turn $\bf{A}$ and $\bf{A}^{-1}$ into Python code

In [20]:
py_A = sym.pycode(A.subs(a for a in zip(Γ, Γ_py))).replace('ImmutableDenseMatrix', 'np.array')
py_Ai = sym.pycode(Ai.subs(a for a in zip(Γ, Γ_py))).replace('ImmutableDenseMatrix', 'np.array')

# View the results
print(f"The A matrix: {py_A}")
print(f"The Ai matrix: {py_Ai}")



The A matrix: np.array([[math.sin(β), 0, 1], [-math.sin(γ)*math.cos(β), math.cos(γ), 0], [math.cos(β)*math.cos(γ), math.sin(γ), 0]])
The Ai matrix: np.array([[0, -math.sin(γ)/math.cos(β), math.cos(γ)/math.cos(β)], [0, math.cos(γ), math.sin(γ)], [1, math.sin(γ)*math.tan(β), -math.cos(γ)*math.tan(β)]])


We can then copy and paste these into Python methods

In [21]:
def JA(Γ):
    """
    Returns a Jacobian which relates angular velocity to
    xyz roll-pitch-yaw rates.

    Γ is a 3 vector representing the current RPY angles
    """
    α, β, γ = Γ

    return np.array([
        [math.sin(β), 0, 1],
        [-math.sin(γ)*math.cos(β), math.cos(γ), 0],
        [math.cos(β)*math.cos(γ), math.sin(γ), 0]
    ])

def JAi(Γ):
    """
    Returns a Jacobian which relates xyz roll-pitch-yaw rates
    to angular velocity.

    Γ is a 3 vector representing the current RPY angles
    """
    α, β, γ = Γ

    return np.array([
        [0, -math.sin(γ)/math.cos(β), math.cos(γ)/math.cos(β)],
        [0, math.cos(γ), math.sin(γ)],
        [1, math.sin(γ)*math.tan(β), -math.cos(γ)*math.tan(β)]
    ])

Lets test these methods out

In [22]:
# Set up our example xyz roll-pitch-yaw vector
ex_Γ = [0.1, -0.4, 0.3]

# Set up our example xyz roll-pitch-yaw rate vector
ex_Γd = [-0.2, -0.1, 0.2]

# Using the JA method, we can convert roll-pitch-yaw rates into
# angular velocity
ex_ω = JA(ex_Γ) @ ex_Γd

# Using the JAi method, we can convert the angular velocity
# back into roll-pitch-yaw rates
ex_Γd_rev = JAi(ex_Γ) @ ex_ω

# View the result
ds.display(ds.Math(f"ω = {vlatex(sym.Matrix(np.round(ex_ω, 2)))}"))
ds.display(ds.Math(f"\dot{{Γ}} = {vlatex(sym.Matrix(np.round(ex_Γd_rev, 2)))}"))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<br>

<a id='acc'></a>

### 3.4 Second Derivatives of $\bf{SO}(3)$ and the Angular Hessian
---

In this section, we are going to define a method which relates some angular acceleration unit to angular velocity. In this case we are choosing xyz roll-pitch-yaw angle accelerations. Using the same techniques presented in this Notebook, you can modify this section to relate different angular acceleration units if desired.

We have the following formula for the derivative of an $\bf{SO}(3)$ object

\begin{align*}
    \dot{\bf{R}}
    &=
    [\omega]_\times \
    \bf{R}
\end{align*}

Using the product and chain rule, we can take the second derivative

\begin{align*}
    \ddot{\bf{R}}
    &= 
    [\dot{\omega}]_\times \
    [\omega]_\times \
    e^{[\eta]_\times}
    +
    [\omega]_\times \
    [\omega]_\times \
    e^{[\eta]_\times}
    \\
    &= 
    [\alpha]_\times \
    \bf{R}
    +
    [\omega]_\times \
    \dot{\bf{R}}
\end{align*}
where we have unfortunately doubled up on our use of $\alpha$ (previously used in the roll-pitch-yaw vector $\Gamma$). In this context $\alpha = (\alpha_x, \ \alpha_y, \ \alpha_z)$ describes the angular acceleration. In a similar theme to $\omega$, the angular acceleration describes an axis of rotation, and acceleration of rotation about that axis.

As we did for the first derivative, lets take the derivative of $\dot{\bf{R}}$ using the xyz roll-pitch-yaw definition

\begin{align*}
    \ddot{\bf{R}}
    &= 
    \ddot{\bf{R}_x}
    \bf{R}_y
    \bf{R}_y
    +
    \dot{\bf{R}_x}
    \dot{\bf{R}_y}
    \bf{R}_y
    +
    \dot{\bf{R}_x}
    \bf{R}_y
    \dot{\bf{R}_y} \\
    %
    & \quad +
    %
    \dot{\bf{R}_x}
    \dot{\bf{R}_y}
    \bf{R}_y
    +
    \bf{R}_x
    \ddot{\bf{R}_y}
    \bf{R}_y
    +
    \bf{R}_x
    \dot{\bf{R}_y}
    \dot{\bf{R}_y} \\
    %
    & \quad +
    %
    \dot{\bf{R}_x}
    \bf{R}_y
    \dot{\bf{R}_y}
    +
    \bf{R}_x
    \dot{\bf{R}_y}
    \dot{\bf{R}_y}
    +
    \bf{R}_x
    \bf{R}_y
    \ddot{\bf{R}_y} \\
    &=
    \ddot{\bf{R}_x}
    \bf{R}_y
    \bf{R}_y
    +
    \bf{R}_x
    \ddot{\bf{R}_y}
    \bf{R}_y
    +
    \bf{R}_x
    \bf{R}_y
    \ddot{\bf{R}_y} +
    2 \left(
    \dot{\bf{R}_x}
    \dot{\bf{R}_y}
    \bf{R}_y
    +
    \dot{\bf{R}_x}
    \bf{R}_y
    \dot{\bf{R}_y}
    +
    \bf{R}_x
    \dot{\bf{R}_y}
    \dot{\bf{R}_y}
    \right)
\end{align*}

Now to work out $\ddot{\bf{R}_x}$ lets look at the definition of $\dot{\bf{R}}$

\begin{align*}
    \dot{\bf{R}}_x
    &= 
    \dot{\gamma} g_x \
    e^{\gamma  g_x} \\
\end{align*}

and take the second derivative using the product and chain rule

\begin{align*}
    \ddot{\bf{R}}_x
    &= 
    \ddot{\gamma} g_x \
    e^{\gamma  g_x} 
    +
    \dot{\gamma} g_x \
    \dot{\gamma} g_x \
    e^{\gamma \ g_x} \\
    &= 
    \ddot{\gamma} g_x \
    \bf{R}_x
    +
    \dot{\gamma} g_x \
    \dot{\bf{R}_x}
\end{align*}

Lets try this out in Python before going any further

In [23]:
# Calculate the derivative of Rx
ddRx = (γddt * gx) @ Rx + (γdt * gx) @ dRx

# View the result
ds.display(ds.Math(f"\ddot{{R}}_x = {vlatex(sym.Matrix(ddRx))}"))

<IPython.core.display.Math object>

To validate our method, lets use `sympy` symbolic derivative function to compare results

In [24]:
# Use symbolic differentiation
ddRx_sym = sym.diff(sym.Matrix(dRx))

# View the sresult
ds.display(ds.Math(f"\ddot{{R}}_x = {vlatex(sym.Matrix(ddRx_sym))}"))

<IPython.core.display.Math object>

Lets calculate the derivatives for Ry and Rz in the same manner before going any further

In [25]:
# Calculate the derivative of dRy and dRz
ddRy = (βddt * gy) @ Ry + (βdt * gy) @ dRy
ddRz = (αddt * gz) @ Rz + (αdt * gz) @ dRz

# View the result
ds.display(ds.Math(f"\ddot{{R}}_y = {vlatex(sym.Matrix(ddRy))}"))
ds.display(ds.Math(f"\ddot{{R}}_z = {vlatex(sym.Matrix(ddRz))}"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Now recall our derivative of $\bf{R}(\Gamma)$ previously defined

\begin{align*}
    \ddot{\bf{R}}
    &=
    \ddot{\bf{R}_x}
    \bf{R}_y
    \bf{R}_y
    +
    \bf{R}_x
    \ddot{\bf{R}_y}
    \bf{R}_y
    +
    \bf{R}_x
    \bf{R}_y
    \ddot{\bf{R}_y} +
    2 \left(
    \dot{\bf{R}_x}
    \dot{\bf{R}_y}
    \bf{R}_y
    +
    \dot{\bf{R}_x}
    \bf{R}_y
    \dot{\bf{R}_y}
    +
    \bf{R}_x
    \dot{\bf{R}_y}
    \dot{\bf{R}_y}
    \right)
\end{align*}

We now have the necessary components to calculate this in Python

In [26]:
ddR = (
    ddRx @ Ry @ Rz
    + Rx @ ddRy @ Rz
    + Rx @ Ry @ ddRz
    + 2 * (dRx @ dRy @ Rz + dRx @ Ry @ dRz + Rx @ dRy @ dRz)
)

# View the result
ds.display(ds.Math(f"\ddot{{R}} = {vlatex(sym.Matrix(ddR))}"))


<IPython.core.display.Math object>

Now using the identity we derived previously
\begin{align*}
    \ddot{\bf{R}}
    &= 
    [\alpha]_\times \
    \bf{R}
    +
    [\omega]_\times \
    \dot{\bf{R}}
\end{align*}

we can rearrange to obtain the angular acceleration $\alpha$

\begin{align*}
    \ddot{\bf{R}} -
    [\omega]_\times \
    \dot{\bf{R}}
    &= 
    [\alpha]_\times \
    \bf{R}
    \\
    \left(
    \ddot{\bf{R}} -
    [\omega]_\times \
    \dot{\bf{R}}
    \right)
    \dot{\bf{R}}^\top
    &= 
    [\alpha]_\times
    \\
    \vee_\times
    \Bigg(
    \left(
    \ddot{\bf{R}} -
    [\omega]_\times \
    \dot{\bf{R}}
    \right)
    \dot{\bf{R}}^\top
    \Bigg)
    &= 
    \alpha
\end{align*}

Now lets try in Python

In [27]:
ωd = vex((ddR - skew(ω.tolist()) @ dR) @ R.T)

# Simplify the result to make it more readable
ωd = sym.simplify(ωd)

# View the result
ds.display(ds.Math(f"\dot{{ω}} = {vlatex(sym.Matrix(ωd))}"))

<IPython.core.display.Math object>

In [28]:
ωd_sym = sym.diff(sym.Matrix(ω))

# Simplify the result to make it more readable
ωd_sym = sym.simplify(ωd_sym)

# View the result
ds.display(ds.Math(f"\dot{{ω}} = {vlatex(sym.Matrix(ωd_sym))}"))

<IPython.core.display.Math object>

We now have a relationship between angular acceleration and roll-pitch-yaw accelerations.

From Section 3.3, we have the following relationship

\begin{align*}
    \omega =
    \bf{A}(\Gamma)
    \dot{\Gamma}
\end{align*}

Taking the derivative of this using the product rule we get

\begin{align*}
    \alpha =
    \dot{\bf{A}}(\Gamma)
    \dot{\Gamma}
    +
    \bf{A}(\Gamma)
    \ddot{\Gamma}
\end{align*}

With this in mind, lets try to re-factor the matrix stored in `ωd` into the form above

In [29]:
# Since we already know A, lets extract Ad from ωd

# Using Algebra, we can find the vector Ad @ Γd = ωd - (Ad @ Γdd)
AdΓd = sym.Matrix(ωd) - A @ sym.Matrix(Γdd)
ds.display(ds.Math(f"\dot{{A}} \\times \ddot{{Γ}} = {vlatex(sym.Matrix(AdΓd))}"))

# Now lets factor out Γd from AdΓd to obtain Ad
Ad = sym.Matrix.zeros(3, 3)

for i in range(3):
    e = ωd[i].expand()
    for j in range(3):
        Ad[i, j] = e.coeff(Γd[j])
        # Remove that term from the e expression
        e = sym.simplify(e - Ad[i, j] * Γd[j])

# View the result
ds.display(ds.Math(f"\dot{{A}} = {vlatex(sym.Matrix(Ad))}"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Note that $\dot{\bf{A}}$ is actually a tensor $\in \mathbb{R}^{3 \times 3 \times 3}$ multiplied by $\Gamma$ to form a matrix $\in \mathbb{R}^{3 \times 3}$.

With this we can write the full equation.

In [30]:
# View in expanded form in Python
ds.display(
    ds.Math(
        f"α = {vlatex(sym.Array(Ad))} {vlatex(sym.Matrix(Γd))} + {vlatex(sym.Matrix(A))}{vlatex(sym.Matrix(Γdd))}"
    )
)

<IPython.core.display.Math object>

Next, we create the $\dot{\bf{A}}^{-1}$ matrix

In [31]:
# unfortunately synpy cannot invert out Ad matrix. We will cheat and use symbolic
# differentiation on the Ai matrix to obtain the derivative of the inverse of A
Adi = sym.trigsimp(sym.diff(sym.Matrix(Ai)))

# View the result
ds.display(ds.Math(f"\dot{{A}}^{{-1}} = {vlatex(sym.Matrix(Adi))}"))

<IPython.core.display.Math object>

Lets turn $\dot{\bf{A}}$ and $\dot{\bf{A}}^{-1}$ into Python code

In [32]:
py_Ad = sym.pycode(
    Ad.subs(a for a in zip(Γd, Γd_py)).subs(a for a in zip(Γ, Γ_py))
).replace("ImmutableDenseMatrix", "np.array")
py_Adi = sym.pycode(
    Adi.subs(a for a in zip(Γd, Γd_py)).subs(a for a in zip(Γ, Γ_py))
).replace("ImmutableDenseMatrix", "np.array")

# View the results
print(f"The Ad matrix: {py_Ad}")
print(f"The Adi matrix: {py_Adi}")


The Ad matrix: np.array([[βd*math.cos(β), 0, 0], [βd*math.sin(β)*math.sin(γ) - γd*math.cos(β)*math.cos(γ), -γd*math.sin(γ), 0], [-βd*math.sin(β)*math.cos(γ) - γd*math.sin(γ)*math.cos(β), γd*math.cos(γ), 0]])
The Adi matrix: np.array([[0, -(βd*math.sin(β)*math.sin(γ)/math.cos(β) + γd*math.cos(γ))/math.cos(β), (βd*math.sin(β)*math.cos(γ)/math.cos(β) - γd*math.sin(γ))/math.cos(β)], [0, -γd*math.sin(γ), γd*math.cos(γ)], [0, βd*math.sin(γ)/math.cos(β)**2 + γd*math.cos(γ)*math.tan(β), -βd*math.cos(γ)/math.cos(β)**2 + γd*math.sin(γ)*math.tan(β)]])


We can then copy and paste these into Python methods

In [33]:
def JAd(Γ, Γd):
    """
    Returns the derivative of the Jacobian which relates
    angular velocity to xyz roll-pitch-yaw rates.

    Γ is a 3 vector representing the current RPY angles
    Γd is a 3 vector representing the current RPY angle rates
    """
    α, β, γ = Γ
    αd, βd, γd = Γd

    return np.array(
        [
            [βd * math.cos(β), 0, 0],
            [
                βd * math.sin(β) * math.sin(γ) - γd * math.cos(β) * math.cos(γ),
                -γd * math.sin(γ),
                0,
            ],
            [
                -βd * math.sin(β) * math.cos(γ) - γd * math.sin(γ) * math.cos(β),
                γd * math.cos(γ),
                0,
            ],
        ]
    )


def JAdi(Γ, Γd):
    """
    Returns the derivative of the Jacobian which relates xyz
    roll-pitch-yaw rates to angular velocity.

    Γ is a 3 vector representing the current RPY angles
    Γd is a 3 vector representing the current RPY angle rates
    """
    α, β, γ = Γ
    αd, βd, γd = Γd

    return np.array(
        [
            [
                0,
                -(βd * math.sin(β) * math.sin(γ) / math.cos(β) + γd * math.cos(γ))
                / math.cos(β),
                (βd * math.sin(β) * math.cos(γ) / math.cos(β) - γd * math.sin(γ))
                / math.cos(β),
            ],
            [0, -γd * math.sin(γ), γd * math.cos(γ)],
            [
                0,
                βd * math.sin(γ) / math.cos(β) ** 2 + γd * math.cos(γ) * math.tan(β),
                -βd * math.cos(γ) / math.cos(β) ** 2 + γd * math.sin(γ) * math.tan(β),
            ],
        ]
    )


<br>

<a id='Ja'></a>

### 3.5 The Analytic Jacobian
---

The kinematic derivatives we have presented in previous Notebooks so far have been in geometric form with translational velocity $\vec{v}$ and angular velocity $\vec{\omega}$ vectors. Some applications require the manipulator Jacobian and further derivatives to to be expressed with different orientation rate representations such as the rate of change of Euler angles, roll-pitch-yaw angles or exponential coordinates -- these are called analytical forms.

In the first four Sections of this Notebook, we have outlined how to translate angular velocity and acceleration into roll-pitch-yaw rates and accelerations. In this section we are going to exploit this relationship to obtain the analytic manipulator Jacobian.


The analytical Jacobian represented with roll-pitch-yaw angle rates is

\begin{align*}
    \bf{J}_a(\bf{q})
    &=
    \tilde{\bf{J}}(\bf{\Gamma}) \ \bf{J}(\bf{q})
    \\
    &=
    \begin{pmatrix}
        \bf{1}_{3 \times 3} & \bf{0}_{3 \times 3} \\
        \bf{0}_{3 \times 3} & \bf{A}^{-1}(\bf{\Gamma}) \\
    \end{pmatrix}
    \bf{J}(\bf{q}).
\end{align*}

Lets turn this into a python method

In [34]:
def jacob0_rpy(robot, q):
    """
    Returns the analytic base-frame manipulator Jacobian where
    the rotation units are expressed as xyz roll-pitch-yaw rates

    robot is an Robot object representing a robot
    q is the current joint coordinates of the robot
    """

    # Calculate the forward kinematics of the robot
    T = robot.fkine(q)

    # Calculate the rpy of the orientation within T
    Γ = T.rpy(order='xyz')

    # Calculate the Jacobian which relates angular velocity to
    # rpy rates
    Ai = JAi(Γ)

    # Construct the augmented Jacobian
    Jaug = np.zeros((6, 6))

    # Top left corner is an identity
    Jaug[:3, :3] = np.eye(3)

    # Bottom right corner is Ai
    Jaug[3:, 3:] = Ai

    # Calculate the base-frame manipulator Jacobian
    J0 = robot.jacob0(q)

    # Calculate the analytic Jacobian
    Ja = Jaug @ J0

    return Ja

Lets test it out

In [35]:
# Make a robot
robot = rtb.models.Panda()

# Set the joint angles
q = robot.qr.copy()

# Calculate the analytic Jacobian
Ja = jacob0_rpy(robot, q)

# View the result
print("Ja using our method:")
print(np.round(Ja, 2))

# Compare to using the roboticstoolbox method
print("\nJa using the toolbox:")
Ja_rtb = robot.jacob0_analytical(q, representation="rpy/xyz")
print(np.round(Ja_rtb, 2))



Ja using our method:
[[ 0.    0.08  0.    0.25  0.    0.2   0.  ]
 [ 0.48  0.    0.49  0.    0.15 -0.    0.  ]
 [-0.   -0.48  0.    0.5  -0.    0.11  0.  ]
 [-1.01 -0.   -0.96  0.    0.32  0.    1.  ]
 [ 0.   -1.    0.    1.    0.    1.   -0.  ]
 [ 0.1  -0.   -0.2  -0.    0.91 -0.    0.  ]]

Ja using the toolbox:
[[ 0.    0.08  0.    0.25  0.    0.2   0.  ]
 [ 0.48  0.    0.49  0.    0.15 -0.    0.  ]
 [-0.   -0.48  0.    0.5  -0.    0.11  0.  ]
 [-1.01 -0.   -0.96  0.    0.32  0.    1.  ]
 [ 0.   -1.    0.    1.    0.    1.   -0.  ]
 [ 0.1  -0.   -0.2  -0.    0.91 -0.    0.  ]]


Using the `robot.jacob0_analytical` method within the `roboticstoolbox` we can calculate the analytic Jacobian in many forms:

        ==================   ==================================
         representation           Rotational representation
        ==================   ==================================
         'rpy/xyz'            RPY angular rates in XYZ order
         'rpy/zyx'            RPY angular rates in XYZ order
         'eul'                Euler angular rates in ZYZ order
         'exp'                exponential coordinate rates
        ==================   ==================================

In [36]:
print("Ja (rpy/xyz):")
Ja_rtb1 = robot.jacob0_analytical(q, representation="rpy/xyz")
print(np.round(Ja_rtb1, 2))

print("\nJa (rpy/zyx):")
Ja_rtb2 = robot.jacob0_analytical(q, representation="rpy/zyx")
print(np.round(Ja_rtb2, 2))

print("\nJa (eul):")
Ja_rtb3 = robot.jacob0_analytical(q, representation="eul")
print(np.round(Ja_rtb3, 2))

print("\nJa (exp):")
Ja_rtb4 = robot.jacob0_analytical(q, representation="exp")
print(np.round(Ja_rtb4, 2))

Ja (rpy/xyz):
[[ 0.    0.08  0.    0.25  0.    0.2   0.  ]
 [ 0.48  0.    0.49  0.    0.15 -0.    0.  ]
 [-0.   -0.48  0.    0.5  -0.    0.11  0.  ]
 [-1.01 -0.   -0.96  0.    0.32  0.    1.  ]
 [ 0.   -1.    0.    1.    0.    1.   -0.  ]
 [ 0.1  -0.   -0.2  -0.    0.91 -0.    0.  ]]

Ja (rpy/zyx):
[[ 0.    0.08  0.    0.25  0.    0.2   0.  ]
 [ 0.48  0.    0.49  0.    0.15 -0.    0.  ]
 [-0.   -0.48  0.    0.5  -0.    0.11  0.  ]
 [-0.   -0.   -0.3  -0.    0.95 -0.    0.1 ]
 [-0.    1.    0.   -1.   -0.   -1.   -0.  ]
 [ 1.    0.    0.98  0.   -0.42  0.   -1.01]]

Ja (eul):
[[ 0.    0.08  0.    0.25  0.    0.2   0.  ]
 [ 0.48  0.    0.49  0.    0.15 -0.    0.  ]
 [-0.   -0.48  0.    0.5  -0.    0.11  0.  ]
 [ 1.   -0.   -1.99  0.    9.11  0.    0.  ]
 [-0.    1.   -0.   -1.    0.   -1.    0.  ]
 [-0.   -0.   -2.96  0.    9.48  0.    1.  ]]

Ja (exp):
[[ 0.    0.08  0.    0.25  0.    0.2   0.  ]
 [ 0.48  0.    0.49  0.    0.15 -0.    0.  ]
 [-0.   -0.48  0.    0.5  -0.    0.11  0.  ]
 

<br>

<a id='Jad'></a>

### 3.6 Derivative of the Analytic Jacobian
---

The derivative of $\bf{J}_a(\bf{q})$ is typically used in applications which have a task-space acceleration term. We can obtain $\dot{\bf{J}}_a(\bf{q})$ from the definition of $\bf{J}_a(\bf{q})$ using the product rule

\begin{align*} 
    \dot{\bf{J}}_a(\bf{q})
    &=
    \dfrac{\text{d} \tilde{\bf{J}}(\bf{\Gamma})}
          {\text{d} t} \
    \bf{J}(\bf{q}) +
    \tilde{\bf{J}}(\bf{\Gamma}) \ 
    \dfrac{\text{d} \bf{J}(\bf{q})}
          {\text{d} t} \\
    &=
    \dfrac{\text{d} \tilde{\bf{J}}(\bf{\Gamma})}
          {\text{d} t} \
    \bf{J}(\bf{q}) +
    \tilde{\bf{J}}(\bf{\Gamma}) \ 
    \big(
        \bf{H}(\bf{q}) \dot{\bf{q}}
    \big)
\end{align*}

Taking the derivative of $\tilde{\bf{J}}(\bf{\Gamma})$ gives

\begin{align*}
    \dfrac{\text{d} \tilde{\bf{J}}(\bf{\Gamma})}
          {\text{d} t}
    &=
    \begin{pmatrix}
        \bf{0}_{3 \times 3} & \bf{0}_{3 \times 3} \\
        \bf{0}_{3 \times 3} & \dot{\bf{A}}^{-1}(\bf{\Gamma, \ \dot{\Gamma}}) \\
    \end{pmatrix}
\end{align*}

Lets turn this into a python method

In [37]:
def jacob0dot_rpy(robot, q, qd):
    """
    Returns the analytic base-frame manipulator Jacobian where
    the rotation units are expressed as xyz roll-pitch-yaw rates

    robot is an Robot object representing a robot
    q is the current joint coordinates of the robot
    """

    # Calculate the forward kinematics of the robot
    T = robot.fkine(q)

    # Calculate the base-frame manipulator Jacobian
    J0 = robot.jacob0(q)

    # Calculate the base-frame manipulator Hessian
    H0 = robot.hessian0(q)

    # Calculate the end-effector velocity and extract angular velocity
    ev = J0 @ qd
    eω = ev[3:]

    # Calculate the rpy of the orientation within T
    Γ = T.rpy(order='xyz')

    # Calculate the rpy rate
    Γd = JAi(Γ) @ eω

    # Calculate the Jacobian which relates angular velocity to
    # rpy rates
    Ai = JAi(Γ)

    # Calculate the derivative of theJacobian which relates angular
    # velocity to rpy rates
    Adi = JAdi(Γ, Γd)

    # Construct the augmented Jacobian
    Jaug = np.zeros((6, 6))

    # Top left corner is an identity
    Jaug[:3, :3] = np.eye(3)

    # Bottom right corner is Ai
    Jaug[3:, 3:] = Ai

    # Construct the augmented Jacobian derivative
    Jaugd = np.zeros((6, 6))

    # Bottom right corner is Ai
    Jaugd[3:, 3:] = Adi

    # Calculate the analytic Jacobian derivative
    Jad = Jaugd @ J0 + Jaug @ np.tensordot(H0, qd, (0, 0))

    return Jad

Lets test it out

In [38]:
# Make a robot
robot = rtb.models.Panda()

# Set the joint angles
q = robot.qr.copy()

# Set the joint rates
qd = np.array([0.1, -0.2, 0.3, -0.1, 0.2, -0.3, 0.4])

# Calculate the analytic Jacobian derivative
Jad = jacob0dot_rpy(robot, q, qd)

# View the result
print("Jad using our method:")
print(np.round(Jad, 2))

# Compare to using the roboticstoolbox method
print("\nJad using the toolbox:")
Jad_rtb = robot.jacob0_dot(q, qd, representation="rpy/xyz")
print(np.round(Jad_rtb, 2))

Jad using our method:
[[-0.23  0.01 -0.22 -0.02 -0.05  0.02  0.  ]
 [-0.1   0.01 -0.11  0.14  0.05  0.05  0.  ]
 [-0.    0.1  -0.05 -0.04  0.02 -0.04  0.  ]
 [ 0.02  0.13  0.08 -0.22 -0.1  -0.03  0.  ]
 [-0.13 -0.   -0.1   0.   -0.29  0.    0.  ]
 [-0.2  -0.11 -0.39  0.41  0.11  0.33 -0.  ]]

Jad using the toolbox:
[[-0.23  0.01 -0.22 -0.02 -0.05  0.02  0.  ]
 [-0.1   0.01 -0.11  0.14  0.05  0.05  0.  ]
 [ 0.    0.1  -0.05 -0.04  0.02 -0.04  0.  ]
 [ 0.02  0.13  0.08 -0.22 -0.1  -0.03 -0.  ]
 [-0.13  0.   -0.1  -0.   -0.29 -0.    0.  ]
 [-0.2  -0.11 -0.39  0.41  0.11  0.33  0.  ]]


Using the `robot.jacob0_dot` method within the `roboticstoolbox` we can calculate the derivative of the analytic Jacobian in many forms using the `representation` kwarg:

        ==================   ==================================
         representation           Rotational representation
        ==================   ==================================
         'rpy/xyz'            RPY angular rates in XYZ order
         'rpy/zyx'            RPY angular rates in XYZ order
         'eul'                Euler angular rates in ZYZ order
         'exp'                exponential coordinate rates
        ==================   ==================================

In [39]:
print("Jad (rpy/xyz):")
Jad_rtb1 = robot.jacob0_dot(q, qd, representation="rpy/xyz")
print(np.round(Jad_rtb1, 2))

print("\nJad (rpy/zyx):")
Jad_rtb2 = robot.jacob0_dot(q, qd, representation="rpy/zyx")
print(np.round(Jad_rtb2, 2))

print("\nJad (eul):")
Jad_rtb3 = robot.jacob0_dot(q, qd, representation="eul")
print(np.round(Jad_rtb3, 2))

print("\nJad (exp):")
Jad_rtb4 = robot.jacob0_dot(q, qd, representation="exp")
print(np.round(Jad_rtb4, 2))

Jad (rpy/xyz):
[[-0.23  0.01 -0.22 -0.02 -0.05  0.02  0.  ]
 [-0.1   0.01 -0.11  0.14  0.05  0.05  0.  ]
 [ 0.    0.1  -0.05 -0.04  0.02 -0.04  0.  ]
 [ 0.02  0.13  0.08 -0.22 -0.1  -0.03 -0.  ]
 [-0.13  0.   -0.1  -0.   -0.29 -0.    0.  ]
 [-0.2  -0.11 -0.39  0.41  0.11  0.33  0.  ]]

Jad (rpy/zyx):
[[-0.23  0.01 -0.22 -0.02 -0.05  0.02  0.  ]
 [-0.1   0.01 -0.11  0.14  0.05  0.05  0.  ]
 [ 0.    0.1  -0.05 -0.04  0.02 -0.04  0.  ]
 [ 0.   -0.19 -0.19  0.48  0.01  0.41 -0.2 ]
 [ 0.   -0.   -0.06  0.    0.42  0.    0.14]
 [-0.    0.02 -0.1   0.04  0.28 -0.14  0.02]]

Jad (eul):
[[-2.300e-01  1.000e-02 -2.200e-01 -2.000e-02 -5.000e-02  2.000e-02
   0.000e+00]
 [-1.000e-01  1.000e-02 -1.100e-01  1.400e-01  5.000e-02  5.000e-02
   0.000e+00]
 [ 0.000e+00  1.000e-01 -5.000e-02 -4.000e-02  2.000e-02 -4.000e-02
   0.000e+00]
 [-0.000e+00  1.221e+01 -7.890e+00 -9.260e+00  1.941e+01 -1.009e+01
   0.000e+00]
 [ 0.000e+00 -0.000e+00  3.600e-01  0.000e+00 -9.200e-01  0.000e+00
   0.000e+00]
 [ 0.